# Simple LangChain quickstart.  

## Improt Environement Variables from File
Need to have a .env file with OPENAI_API_KEY

In [ ]:
from environs import Env
env = Env()
env.read_env("/Users/geoffreysmalling/development/langchain/.env")

## connect to OpenAI

In [ ]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(api_key=env.str("OPENAI_API_KEY"))


## Test llm connection and get answer

In [ ]:
llm.invoke("how can langsmith help with testing?")

## create a prompt
use a ChatPromptTemplate to create a prompt

In [ ]:
from langchain_core.prompts import ChatPromptTemplate
prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a world class technical documentation writer."),
    ("user", "{input}")
])

## chain the prompt to the llm
execute the prompt then pass the results to the llm

In [ ]:
chain = prompt | llm #chains the prompt operator and the llm

### Use the Chain of Prompt and LLM to invoke the LLM

In [ ]:
chain.invoke({"input": "how can langsmith help with testing?"})

### LLM responds by default in a Message format object.  
Simply parse the message into a string with LangChain parsers

In [ ]:
from langchain_core.output_parsers import StrOutputParser

output_parser = StrOutputParser()

chain = prompt | llm | output_parser

chain.invoke({"input": "how can langsmith help with testing?"})

## Scrape websits to get content

In [ ]:
from langchain_community.document_loaders import WebBaseLoader
loader = WebBaseLoader("https://docs.smith.langchain.com/user_guide")
docs = loader.load()

In [ ]:
print(type(docs))
print(docs[0])


## Use Embeddings to create vecotrs for searching against

In [ ]:

from langchain_openai import OpenAIEmbeddings

embeddings = OpenAIEmbeddings()

## Store embeddings into Vector Store
Using FAISS (Facebook AI Similarity Search) a simple in memory vector store
first we split the documents
then usting the set of documents and the openai embeddings api we calculate a vector db


In [ ]:
from langchain_community.vectorstores import FAISS
from langchain_text_splitters import RecursiveCharacterTextSplitter


text_splitter = RecursiveCharacterTextSplitter()
documents = text_splitter.split_documents(docs)
print(type(documents))
print(len(docs))
print(len(documents))
vector = FAISS.from_documents(documents, embeddings)

In [ ]:
print(vector)

## Search the vector store using similarity search

In [ ]:
result_docs = vector.similarity_search('monitoring')

In [ ]:
print(result_docs[0])

# create a new prompt with context

In [ ]:
from langchain.chains.combine_documents import create_stuff_documents_chain

prompt = ChatPromptTemplate.from_template("""Answer the following question based only on the provided context:

<context>
{context}
</context>

Question: {input}""")

document_chain = create_stuff_documents_chain(llm, prompt)

from langchain_core.documents import Document

document_chain.invoke({
    "input": "how can langsmith help with testing?",
    "context": [Document(page_content="langsmith can let you visualize test results")]
})


# using a retriever
Langchain abstracts the retrieval process using a retrieval chain
pass in a retriever using the vector store
and the document chain

In [ ]:
from langchain.chains import create_retrieval_chain

retriever = vector.as_retriever()
retrieval_chain = create_retrieval_chain(retriever, document_chain)

In [ ]:
response = retrieval_chain.invoke({"input": "how can langsmith help with testing?"})
print(response["answer"])

In [ ]:
print(response)

In [ ]:
print(response.keys())

### we can attrbute the source by traversing to the metadata object

In [ ]:
print(response['context'][0].metadata['source'])